# Processamento de dados usando o Databricks - PySpark e Spark SQL
  * Este notebook tem como objetivo processar um dataset do Kaggle (https://www.kaggle.com/datasets/lfarhat/brasil-students-scholarship-prouni-20052019) aplicando algumas funções de PySpark e Spark SQL
  * Ao longo do laboratório algumas transformações serão feitas neste dataset e também algumas perguntas serão respondidas.

In [0]:
# Importando as bibliotecas que serão usadas no laboratório.

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Importando o arquivo csv para processamento

In [0]:
# Tipo do arquivos
file_type = "csv"

# Path dos arquivos
path = '/FileStore/tables/prouni_2005_2019.csv'

arq_prouni = spark \
             .read.format(file_type) \
             .option("inferSchema","True") \
             .option("header","True") \
             .csv(path)

In [0]:
%fs ls /FileStore/tables/prouni_2005_2019.csv

path,name,size,modificationTime
dbfs:/FileStore/tables/prouni_2005_2019.csv,prouni_2005_2019.csv,430091396,1655243222000


In [0]:
#Quantidade total de linhas

arq_prouni.count()

Out[16]: 2692540

In [0]:
# Verificando o schema das colunas.

arq_prouni.printSchema()

root
 |-- ANO_CONCESSAO_BOLSA: integer (nullable = true)
 |-- CODIGO_EMEC_IES_BOLSA: integer (nullable = true)
 |-- NOME_IES_BOLSA: string (nullable = true)
 |-- TIPO_BOLSA: string (nullable = true)
 |-- MODALIDADE_ENSINO_BOLSA: string (nullable = true)
 |-- NOME_CURSO_BOLSA: string (nullable = true)
 |-- NOME_TURNO_CURSO_BOLSA: string (nullable = true)
 |-- CPF_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- SEXO_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- RACA_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- DT_NASCIMENTO_BENEFICIARIO: timestamp (nullable = true)
 |-- BENEFICIARIO_DEFICIENTE_FISICO: string (nullable = true)
 |-- REGIAO_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- SIGLA_UF_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- MUNICIPIO_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- idade: double (nullable = true)



In [0]:
#Visualizando algumas linhas do dataframe

arq_prouni.show(10)

+-------------------+---------------------+--------------------+-----------------+-----------------------+--------------------+----------------------+----------------------+-----------------------+-----------------------+--------------------------+------------------------------+-------------------------+---------------------------+----------------------------+-----+
|ANO_CONCESSAO_BOLSA|CODIGO_EMEC_IES_BOLSA|      NOME_IES_BOLSA|       TIPO_BOLSA|MODALIDADE_ENSINO_BOLSA|    NOME_CURSO_BOLSA|NOME_TURNO_CURSO_BOLSA|CPF_BENEFICIARIO_BOLSA|SEXO_BENEFICIARIO_BOLSA|RACA_BENEFICIARIO_BOLSA|DT_NASCIMENTO_BENEFICIARIO|BENEFICIARIO_DEFICIENTE_FISICO|REGIAO_BENEFICIARIO_BOLSA|SIGLA_UF_BENEFICIARIO_BOLSA|MUNICIPIO_BENEFICIARIO_BOLSA|idade|
+-------------------+---------------------+--------------------+-----------------+-----------------------+--------------------+----------------------+----------------------+-----------------------+-----------------------+--------------------------+--------------

#Gerando cache dos DataFrames
O objetivo é facilitar o processamento uma vez que estaremos realizando algumas operações neles.

In [0]:
arq_prouni.cache();

#Criando uma tabela temporia SQL

In [0]:
arq_prouni.createOrReplaceTempView("TempProuni")

In [0]:
#Tipando a coluna 'idade' e 'data_nascimento'. 
#Criando um novo DataFrame apartir de uma query SQL

df_sql_full = spark.sql("""
            select
            
             ANO_CONCESSAO_BOLSA,
             CODIGO_EMEC_IES_BOLSA,
             NOME_IES_BOLSA,
             TIPO_BOLSA,
                 CASE WHEN TIPO_BOLSA == 'BOLSA INTEGRAL' THEN '1'
                     WHEN TIPO_BOLSA == 'BOLSA PARCIAL 25%' THEN '2'
                         WHEN TIPO_BOLSA == 'BOLSA PARCIAL 50%' THEN '3'
                             END AS NR_TIPO_BOLSA,
             MODALIDADE_ENSINO_BOLSA,
             NOME_CURSO_BOLSA,
             NOME_TURNO_CURSO_BOLSA,
             CPF_BENEFICIARIO_BOLSA,
             SEXO_BENEFICIARIO_BOLSA,
             RACA_BENEFICIARIO_BOLSA,
             
                 CAST(DT_NASCIMENTO_BENEFICIARIO as DATE) as DATA_NASCIMENTO,
                 
             BENEFICIARIO_DEFICIENTE_FISICO,
             REGIAO_BENEFICIARIO_BOLSA,
             SIGLA_UF_BENEFICIARIO_BOLSA,
             MUNICIPIO_BENEFICIARIO_BOLSA,
             
                 CAST(idade AS INT) AS IDADE
 
             
             from TempProuni
""")

#Verificando a nova tipagem das colunas
df_sql_full.printSchema()


root
 |-- ANO_CONCESSAO_BOLSA: integer (nullable = true)
 |-- CODIGO_EMEC_IES_BOLSA: integer (nullable = true)
 |-- NOME_IES_BOLSA: string (nullable = true)
 |-- TIPO_BOLSA: string (nullable = true)
 |-- NR_TIPO_BOLSA: string (nullable = true)
 |-- MODALIDADE_ENSINO_BOLSA: string (nullable = true)
 |-- NOME_CURSO_BOLSA: string (nullable = true)
 |-- NOME_TURNO_CURSO_BOLSA: string (nullable = true)
 |-- CPF_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- SEXO_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- RACA_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- DATA_NASCIMENTO: date (nullable = true)
 |-- BENEFICIARIO_DEFICIENTE_FISICO: string (nullable = true)
 |-- REGIAO_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- SIGLA_UF_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- MUNICIPIO_BENEFICIARIO_BOLSA: string (nullable = true)
 |-- IDADE: integer (nullable = true)



#Respondendo algumas perguntas sobre os dados

* Escrever uma query que mostre a quantidade de inscrições por região.

In [0]:
%sql
 
SELECT REGIAO_BENEFICIARIO_BOLSA, count(CPF_BENEFICIARIO_BOLSA)
      FROM TempProuni
WHERE 
REGIAO_BENEFICIARIO_BOLSA <> 'null'
 
GROUP BY
REGIAO_BENEFICIARIO_BOLSA
 
ORDER BY
count(CPF_BENEFICIARIO_BOLSA) DESC
 

* Escrever uma  query que mostre as inscrições por turno, tipo da bolsa e região ?

In [0]:
%sql
 
SELECT REGIAO_BENEFICIARIO_BOLSA, NOME_TURNO_CURSO_BOLSA, TIPO_BOLSA , count(CPF_BENEFICIARIO_BOLSA) AS TOTAL_INSCRICAO
      FROM TempProuni
WHERE 
REGIAO_BENEFICIARIO_BOLSA <> 'null'

GROUP BY
NOME_TURNO_CURSO_BOLSA,
REGIAO_BENEFICIARIO_BOLSA,
TIPO_BOLSA

ORDER BY
count(CPF_BENEFICIARIO_BOLSA) DESC

* Escreva uma query que mostre qual é a quantidade total de inscrições por curso.

In [0]:
%sql

SELECT  NOME_CURSO_BOLSA , count(CPF_BENEFICIARIO_BOLSA) AS TOTAL_INSCRITOS
  FROM TempProuni
GROUP BY
NOME_CURSO_BOLSA

ORDER BY
count(CPF_BENEFICIARIO_BOLSA) DESC

* Qual o total de beneficiários com deficiência física ?

In [0]:
%sql

SELECT  BENEFICIARIO_DEFICIENTE_FISICO,  count(CPF_BENEFICIARIO_BOLSA) AS TOTAL_INSCRITOS
  FROM TempProuni
  WHERE BENEFICIARIO_DEFICIENTE_FISICO in('sim','Sim')

GROUP BY
BENEFICIARIO_DEFICIENTE_FISICO

ORDER BY
count(CPF_BENEFICIARIO_BOLSA) DESC